The purpose of this notebook is to solve the following problem, with the hint that (4,-1,11) is an invalid integer solution:


![fruit](fruit_problem.webp)



In [1]:
R.<a,b,c> = QQ[]

orig_pt = (4,-1,11)
f = a*(a+c)*(a+b) + b*(b+c)*(a+b) + c*(b+c)*(a+c) - 4*(a+b)*(a+c)*(b+c)
print("Equation to solve:", f, "= 0")
print("With known integer point", orig_pt)

#Birationally convert f to an elliptic curve in Weierstrass form
R.<u,v,w> = QQ[]

g=f(u+v,-v,w)

R.<x,y,z> = QQ[]

h = g(z,y-z*1/2,-x)
r = h(x, y, (z-x)/6)

# X = -c
# Y = (a-b)/2
# Z = 6a+6b-c

# a = -1/12X + Y + 1/12Z
# b = -1/12X - Y + 1/12Z
# c = -X
S.<X,Y> = QQ[]
s = r(X,Y,1)

print("This problem is birationally equivalent to", s, "= 0")

Equation to solve: a^3 - 3*a^2*b - 3*a*b^2 + b^3 - 3*a^2*c - 5*a*b*c - 3*b^2*c - 3*a*c^2 - 3*b*c^2 + c^3 = 0
With known integer point (4, -1, 11)
This problem is birationally equivalent to -91/216*X^3 - 95/144*X^2 + Y^2 + 1/12*X - 1/432 = 0


In [2]:
# Define the equivalence explicitly

def to_new (a, b, c):
    z = 6*a+6*b-c
    return (-c/z,(a-b)/(2*z))

def to_orig (x, y):
    a = -x/12 + y + 1/12
    b = -x/12 - y + 1/12
    c = -x
    d = lcm(lcm(a.denominator(), b.denominator()), c.denominator())
    if(a < 0):
        d *= -1
    return (a*d, b*d, c*d)

In [3]:
# f(x,y)=y^2+ax^3+bx^2+cx+d
# is a rational nonsingular elliptic curve
# Compute p+q
# None is used as the point at infinity
def third_pt (f, p,q):
    if q == None:
        return p
    if p == None: 
        return q
    
    #y=ax+b
    a=0
    b=0
    R.<x,y> = QQ[]
    f = f(x,y)
    if q[0] == p[0]:
        if p[1] != q[1]:
            return None
        #p=q, find tangent line
        fx = f.derivative(x)(p[0],p[1])
        fy = f.derivative(y)(p[0],p[1])
        
        a = -fx/fy
        b = (fx*p[0]+fy*p[1])/fy
    else:
        a = (q[1]-p[1])/(q[0]-p[0])
        b = p[1]-a*p[0]
    
    S.<t> = QQ[]
    h = f(t,a*t+b)
    lead = h.lc()
    snd = h.monomial_coefficient(t^2)
    x = -snd/lead - p[0] - q[0]
    y = a*x+b
    return (x,-y)

In [5]:
# We're given a rational point on the curve
orig_pt = (4,-1,11)
rat_pt = to_new(*orig_pt)
assert s(*rat_pt) == 0

# Starting with the point at infinity, keep generating new points until we find a valid solution
points = [None]
pos_point = None

def pos_sol(p):
    return p[0] > 0 and p[1] > 0 and p[2] > 0

# We check multiples of the known rational point and check if they correspond to an integer solution
for i in range(16):
    p = third_pt(s, points[i], rat_pt)
    # p = i * rat_pt
    points.append(p)
    p_orig = to_orig(*p)
    if(pos_sol(p_orig)):
        pos_point = p_orig
        print("Solution Found!")
        print(pos_point)
        break
        
assert f(*pos_point) == 0
print("Found through the following chain:")
print(points)

Solution Found!
(154476802108746166441951315019919837485664325669565431700026634898253202035277999, 36875131794129999827197811565225474825492979968971970996283137471637224634055579, 4373612677928697257861252602371390152816537558161613618621437993378423467772036)
Found through the following chain:
[None, (-11/7, 5/14), (1033/175, -18283/1750), (-2026741/2162342, 37680705/67341508), (69295307953/58496915575, -13227649274463287/10694991074577250), (-40958480975543891/106344534059483167, 8623324751903522349321775/26215233769229919237757094), (314504129718361144991521/802337759501163701306400, -26443434904679357368074637099568813/85172358650336633704021584768576000), (-128131170448868594051599857623171/1143454311979347891037421576622127, 4065195217259461183342765346158535429612565426185/29228679860033230807132710756638779772461972946774), (1745576369976748612001103416037960248713633/10235246577682451732591250786208230357147175, -23974760072348342839771691061558332653671066030766631895076315